# 取得原始資料

## 財報

In [1]:
import requests

links_dict = {"資產負債表": "https://mops.twse.com.tw/mops/web/ajax_t164sb03", 
              "綜合損益表": "https://mops.twse.com.tw/mops/web/ajax_t164sb04", 
              "現金流量表": "https://mops.twse.com.tw/mops/web/ajax_t164sb05", 
              "權益變動表": "https://mops.twse.com.tw/mops/web/ajax_t164sb06"}
 
 
 
statement = input("請輸入想要查詢的報表: ")
stock_id = input("請輸入股票代碼: ")
year = input("請輸入民國年份: ")
# season = input("請輸入第幾季: ")

data = {
    "ncodeURIComponent": "1",
    "step": "1",
    "firstin": "1",
    "off": "1",
    "queryName": "co_id",
    "inpuType": "co_id",
    "TYPEK": "all",
    "isnew": "false",
    "co_id": stock_id,
    "year": year,
    "season": 4 # season
}

response = requests.post(links_dict[statement], data = data)
print(response.text)




<html>
<head>
	<title>公開資訊觀測站</title>
	
	
	<link href="css/css2.css" rel="stylesheet" type="text/css" Media="Screen"/> 
	
	<script type="text/javascript" src="js/mops2.js"></script>
	
	
	
</head>

<body>


<div id="div01" >
<center>
<h2 align='center'><font color='blue'>合併現金流量表</font></h2>
<h4 align='center'><font color='blue'>本資料由台積電公司提供</font></h4>
<table class='noBorder' align='center''><tr><td><font color='red' size='2'>「投資人若需了解更詳細資訊可至<a target='_blank' style='color:violet;font-size:small;font-weight:bold;font-family:Arial, Helvetica, Sans-serif,"宋體";text-decoration:underline;' href='/server-java/t164sb01?step=1&CO_ID=2330&SYEAR=2023&SSEASON=4&REPORT_ID=C'>XBRL資訊平台</a>或<a target='_blank' style='color:violet;font-size:small;font-weight:bold;font-family:Arial, Helvetica, Sans-serif,"宋體";text-decoration:underline;' href='https://doc.twse.com.tw/server-java/t57sb01?step=1&colorchg=1&co_id=2330&year=112&seamon=&mtype=A&'>電子書查詢</a>」</font></td></tr><tr><td><b>本公司採  月制會計年度(空白表曆年制)</b><

In [2]:
import re
from bs4 import BeautifulSoup as bs

soup = bs(response.text, features = "html.parser")
# print(soup.prettify())
description = soup.find("h4").text
start = description.find("由")
end = description.find("公")
company_name = description[start + 1: end]


if statement == "權益變動表":
    first_table = soup.find_all("table")[1]
    second_table = soup.find_all("table")[2]
    html_content = str(first_table) + str(second_table)
else:
    html_content = soup.find_all("table")[1].prettify()

# print(html_content)
html_content = re.sub(r"民國", company_name + "民國", html_content) # \d{3}年第\d季
html_content = html_content.replace("第4季", "")
html_content

'<table align="center" class="hasBorder">\n <tr>\n  <th class="tblHead" colspan="7">\n   台積電民國112年\n  </th>\n </tr>\n <tr>\n  <th class="tblHead" colspan="7" style="text-align:right;">\n   單位：新台幣仟元\n  </th>\n </tr>\n <tr>\n  <th class="tblHead" style="text-align:left;">\n   會計項目\n  </th>\n  <th class="tblHead" style="text-align:left;white-space:nowrap;">\n   112年度\n  </th>\n  <th class="tblHead" style="text-align:left;white-space:nowrap;">\n   111年度\n  </th>\n </tr>\n <tr>\n  <th align="center" class="tblHead">\n  </th>\n  <th align="center" class="tblHead">\n   金額\n  </th>\n  <th align="center" class="tblHead">\n   金額\n  </th>\n </tr>\n <tr>\n  <td class="even" style="text-align:left;white-space:nowrap;">\n   營業活動之現金流量－間接法\n  </td>\n  <td class="even" style="text-align:right;">\n  </td>\n  <td class="even" style="text-align:right;">\n  </td>\n </tr>\n <tr>\n  <td class="odd" style="text-align:left;white-space:nowrap;">\n   繼續營業單位稅前淨利（淨損）\n  </td>\n  <td class="odd" style="text-align:r

In [3]:
from flask import Flask, make_response

app = Flask(__name__)

@app.route('/get_table', methods=['GET'])
def get_table():
    response = make_response(html_content)
    response.mimetype = 'text/html'
    return response

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Dec/2024 11:53:20] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 11:53:28] "GET /get_table HTTP/1.1" 200 -


In [1]:
import http.client
import urllib.parse

conn = http.client.HTTPConnection("127.0.0.1", 5000)

# 將中文字串進行 URL 編碼
params = {
    'stock_id': '6214',
    'year': '2023',
    'statement': '綜合損益表'
}
encoded_params = urllib.parse.urlencode(params)

# 傳送請求
conn.request("GET", f"/get_table?{encoded_params}")
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

<table align="center" class="hasBorder">
 <tr>
  <th class="tblHead" colspan="6">
   精誠民國112年
  </th>
 </tr>
 <tr>
  <th class="tblHead" colspan="6" style="text-align:right;">
   單位：新台幣仟元
  </th>
 </tr>
 <tr>
  <th class="tblHead" style="text-align:left;">
   會計項目
  </th>
  <th class="tblHead" colspan="2" style="text-align:left;white-space:nowrap;">
   112年度
  </th>
  <th class="tblHead" colspan="2" style="text-align:left;white-space:nowrap;">
   111年度
  </th>
 </tr>
 <tr>
  <th align="center" class="tblHead">
  </th>
  <th align="center" class="tblHead">
   金額
  </th>
  <th align="center" class="tblHead">
   %
  </th>
  <th align="center" class="tblHead">
   金額
  </th>
  <th align="center" class="tblHead">
   %
  </th>
 </tr>
 <tr>
  <td class="even" style="text-align:left;white-space:nowrap;">
   銷貨收入
  </td>
  <td class="even" style="text-align:right;">
   27,795,078
  </td>
  <td class="even" style="text-align:right;">
   79.00
  </td>
  <td class="even" style="text-align:right;">
